In [86]:
import glob
import os
import numpy as np
import pandas as pd
from math import isnan

from keras.preprocessing import sequence
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

from sklearn.model_selection import train_test_split

from spacy.lang.en import English

In [2]:
def gather_data(filenames):
    df = pd.read_excel(filenames[0])
    for i in range(1,len(filenames)):
        filename = filenames[i]
        df = pd.concat([df, pd.read_excel(filename)], sort=False)
    return df

In [3]:
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [87]:
# filenames = ['paragraphs_encoded_1.xlsx', 'paragraphs_encoded_2.xlsx']
filenames = glob.glob("Encoded/*")
df = gather_data(filenames)
text = df.Text.to_list()
encodings = df.Sentiment.to_list()
paragraphs = []
labels = []
words = []
for i in range(len(text)):
    paragraph = text[i]
    if type(paragraph) == str:
        label = encodings[i]
        if not isnan(label):
    #         paragraphs.append(paragraph)
    #         try:
    #             labels.append(int(label))
    #         except:
    #             print(i, paragraph)
            if label != 2.:
                paragraphs.append(paragraph)
                if label == 1. or label == 0.:
                    labels.append(0.)
                elif label == 3. or label == 4.:
                    labels.append(1.)
    #         else:
    #             labels.append(0.5)

In [88]:
frames = [frame for frame in df.Frame.to_list() if type(frame) == str]
print('Frames: {}'.format(len(frames)))
print('Paragraphs: {}'.format(len(paragraphs)))

Frames: 654
Paragraphs: 1011


In [89]:
words = list(set([str(word).lower() for paragraph in paragraphs for word in tokenizer(paragraph) if str(word).isalpha()]))
embedding = {}
reverse = {}
for i in range(len(words)):
    word = words[i]
    number = i + 1
    embedding[word] = number
    reverse[number] = word

In [90]:
embedded_paragraphs = [[embedding[str(token).lower()] for token in tokenizer(paragraph) if str(token).lower().isalpha()] for paragraph in paragraphs]

In [91]:
# label_documents = [3,4,1,1,2,2,2,0,3,2,4,2,1,0,4,0,4,4,1,2,3,0,4,2,4,4,0,0,3,2,4,3,1,2,4,0,1,0,4,4,3,4]
# label_documents = [2,2,0,0,1,1,1,0,2,1,2,1,0,0,2,0,2,2,0,1,2,0,2,1,2,2,0,0,2,1,2,2,0,1,2,0,0,0,2,2,2,2]

In [132]:
test_size = 0.4
X_train, X_test, Y_train, Y_test = train_test_split(embedded_paragraphs, labels, test_size=test_size)

In [133]:
print('Average sentence length: {}'.format(round(sum([len(i) for i in embedded_paragraphs])/len(embedded_paragraphs))))
print('Maximum sentence length: {}'.format(round(max([len(i) for i in embedded_paragraphs]))))

Average sentence length: 48
Maximum sentence length: 382


In [134]:
max_words = 100
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [135]:
units = 200
embedding_size=32
vocabulary_size=len(words)+1
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(units, return_sequences=True))
model.add(LSTM(units, return_sequences=True))
model.add(LSTM(units))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 32)           205600    
_________________________________________________________________
lstm_16 (LSTM)               (None, 100, 200)          186400    
_________________________________________________________________
lstm_17 (LSTM)               (None, 100, 200)          320800    
_________________________________________________________________
lstm_18 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 201       
Total params: 1,033,801
Trainable params: 1,033,801
Non-trainable params: 0
_________________________________________________________________
None


In [136]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [137]:
batch_size = 32
num_epochs = 10

X_valid, y_valid = X_train[:batch_size], Y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], Y_train[batch_size:]

model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Train on 574 samples, validate on 32 samples
Epoch 1/10
574/574 [==============================] - 11s 20ms/step - loss: 0.6910 - accuracy: 0.5557 - val_loss: 0.7028 - val_accuracy: 0.4375
Epoch 2/10
574/574 [==============================] - 10s 17ms/step - loss: 0.6800 - accuracy: 0.5662 - val_loss: 0.7019 - val_accuracy: 0.4375
Epoch 3/10
574/574 [==============================] - 10s 17ms/step - loss: 0.5031 - accuracy: 0.7300 - val_loss: 0.5441 - val_accuracy: 0.7500
Epoch 4/10
574/574 [==============================] - 10s 17ms/step - loss: 0.3083 - accuracy: 0.8937 - val_loss: 0.3959 - val_accuracy: 0.7500
Epoch 5/10
574/574 [==============================] - 10s 17ms/step - loss: 0.1254 - accuracy: 0.9425 - val_loss: 0.5084 - val_accuracy: 0.7812
Epoch 6/10
574/574 [==============================] - 10s 18ms/step - loss: 0.0943 - accuracy: 0.9547 - val_loss: 0.4657 - val_accuracy: 0.7812
Epoch 7/10
574/574 [==============================] - 10s 18ms/step - loss: 0.0552 - accura

In [138]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.7530864477157593


In [142]:
# model.save('simplified_acc:{}%_layers:3_units_bs:{}_e:{}_max_w:{}_test_size:{}.h5'.format(round(scores[1],2), units, batch_size, num_epochs, max_words, test_size))

In [140]:
Y_predict = np.round(model.predict(X_test), 2)

In [141]:
for i in range(len(Y_predict)):
    print(Y_predict[i], Y_test[i])
    print([reverse[num] for num in X_test[i] if num != 0])
    print('- - - - - - - - - - - - - - - - - - - - - - - \n')

[0.] 0.0
['pandemic', 'many', 'previously', 'healthy', 'children', 'became', 'critically', 'ill', 'developing', 'severe', 'pneumonia', 'and', 'respiratory', 'failure', 'sometimes', 'fatal', 'the', 'largest', 'nationwide', 'investigation', 'to', 'date', 'of', 'influenza', 'in', 'critically', 'ill', 'children', 'led', 'by', 'children', 'hospital', 'boston', 'found', 'one', 'key', 'risk', 'factor', 'simultaneous', 'infection', 'with', 'methicillin', 'resistant', 'staphylococcus', 'aureus', 'mrsa', 'increased', 'the', 'risk', 'for', 'flu', 'related', 'mortality', 'among', 'previously', 'healthy', 'children', 'moreover', 'almost', 'all', 'of', 'these', 'co', 'infected', 'children', 'were', 'rapidly', 'treated', 'with', 'vancomycin', 'considered', 'to', 'be', 'appropriate', 'treatment', 'for', 'mrsa', 'the', 'fact', 'that', 'they', 'died', 'despite', 'this', 'treatment', 'is', 'especially', 'alarming', 'given', 'the', 'rising', 'rates', 'of', 'mrsa', 'carriage', 'among', 'children', 'in', 't

['the', 'dimensions', 'of', 'the', 'oil', 'droplets', 'are', 'important', 'for', 'a', 'wide', 'range', 'of', 'applications', 'for', 'example', 'the', 'size', 'of', 'the', 'oil', 'droplets', 'determines', 'how', 'creamy', 'a', 'food', 'product', 'tastes', 'as', 'with', 'the', 'current', 'mixing', 'methods', 'the', 'oil', 'droplets', 'produced', 'are', 'not', 'only', 'the', 'large', 'creamy', 'tasting', 'droplets', 'but', 'also', 'less', 'desirable', 'small', 'droplets', 'manufacturers', 'often', 'add', 'slightly', 'more', 'oil', 'than', 'necessary', 'to', 'ensure', 'the', 'desired', 'taste', 'using', 'the', 'chip', 'from', 'wageningen', 'university', 'all', 'oil', 'droplets', 'are', 'exactly', 'the', 'same', 'size', 'this', 'way', 'the', 'same', 'creamy', 'taste', 'can', 'be', 'achieved', 'with', 'less', 'fat']
- - - - - - - - - - - - - - - - - - - - - - - 

[0.99] 0.0
['gaps', 'remain', 'there', 'has', 'been', 'a', 'surge', 'in', 'the', 'number', 'of', 'articles', 'appearing', 'in', 't